An example script for using texturegan model

In [1]:
from argparser import parse_arguments

In [2]:
#dummy command
command = '--display_port 7770 --load 0 --load_D -1 --load_epoch 105 --gpu 2 --model texturegan --feature_weight 1e2 --pixel_weight_ab 1e3 --global_pixel_weight_l 1e3 --local_pixel_weight_l 0 --style_weight 0 --discriminator_weight 1e3 --discriminator_local_weight 1e6  --learning_rate 1e-4 --learning_rate_D 1e-4 --batch_size 36 --save_every 50 --num_epoch 100000 --save_dir /home/psangkloy3/skip_leather_re/ --load_dir /home/psangkloy3/skip_leather_re/ --data_path ../../training_handbags_pretrain/ --learning_rate_D_local  1e-4 --local_texture_size 50 --patch_size_min 20 --patch_size_max 40 --num_input_texture_patch 1 --visualize_every 5 --num_local_texture_patch 1'
args = parse_arguments(command.split())


In [3]:
from main import get_transforms
from dataloader import imfol
from dataloader.imfol import ImageFolder, make_dataset
import torch
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data import DataLoader

## Load from validation folder

In [16]:
args.batch_size = 1
args.image_size =152
args.data_path = '/home/psangkloy3/Patsorn/'

In [17]:
transform = get_transforms(args)
val = make_dataset(args.data_path, 'val')
valDset = ImageFolder('val', args.data_path, transform)
val_display_size = 1
valLoader = DataLoader(dataset=valDset, batch_size=val_display_size,shuffle=True)

## Load Pretrained model

In [11]:
def load_network(model, save_path):
        
    model_state = torch.load(save_path)
    
    if "state_dict" in model_state:
        model.load_state_dict(model_state["state_dict"])
    else:
        model.load_state_dict(model_state)

        model_state = {
            'state_dict': model.cpu().state_dict(),
            'epoch': epoch,
            'iteration': iteration,
            'model': args.model,
            'color_space': args.color_space,
            'batch_size': args.batch_size,
            'dataset': dataset,
            'image_size': args.image_size
        }
    
    model.cuda()


In [12]:
from models import texturegan,discriminator

netG = texturegan.TextureGAN(5, 3, 32)
load_network(netG, '/home/psangkloy3/best_models/textureD_final_allloss_handbag_3300.pth')

netG.eval()

TextureGAN(
  (model): Sequential(
    (main_model): MainModel(
      (main_model): Sequential(
        (conv_1): Conv2d(5, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batch_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_1): ReLU(inplace)
        (res_block_1): ResidualBlock(
          (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (batch_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_2): ReLU(inplace)
        (res_block_2): Residual

In [19]:
data = valLoader.__iter__().__next__()

StopIteration: 

In [18]:
#data_txt = np.load('nice_txt.npy')
#_, _, _, _, txt = data_txt

data = valLoader.__iter__().__next__()

img, skg, seg, eroded_seg, txt = data

StopIteration: 

In [ ]:
MAX = 100
count = 0
results = []
datas = []
datas_txt = []
with torch.cuda.device():    
    for i_ in range(MAX):
    #if True:
        if count> MAX:
            break
        count = count + 1
        data_txt = txtLoader.__iter__().__next__()#datas_txt[32]#
        datas_txt.append(data_txt)
        _, _, _, _, txt = data_txt
        
        #data[1] = nice_skg_shoe[0][1]
        #data[2] = nice_skg_shoe[0][2]
        #data[4] = nice_skg[0][0]#data_handbag[54][4]
        print count-1
        plt.figure(figsize=(20, 8))
        data_temps = []
        temp_results = []
        for i,valLoader in enumerate(loader):
            data = valLoader.__iter__().__next__()
            data_temps.append(data)
            #data = target_data#skgs[i]
            #data[1] = skg_nice[1]
            data[4] = data_txt[4]
            #data[-1] = data[0]
            if i%2==0:#True:#i==0:
                img, skg, seg, eroded_seg, txt = data

                img = custom_transforms.normalize_lab(img)
                skg = custom_transforms.normalize_lab(skg)
                txt = custom_transforms.normalize_lab(txt)
                seg = custom_transforms.normalize_seg(seg)
                eroded_seg = custom_transforms.normalize_seg(eroded_seg)
                inp,texture_loc = get_input(data,-1,-1,30,1)#(data,80,75,30,1)#(data,60,90,30,1)#

                seg = seg!=0
                #print inpv.size()
                #print model[0]
            
            num_model = len(models)
            
            
            #for i,model in enumerate(models):
            model = models[i]

            inpv = get_inputv(inp.cuda())
            output = model(inpv.cuda())

            out_img = vis_image(custom_transforms.denormalize_lab(output.data.double().cpu()),
                                    color_space)
            inp_img = vis_patch(custom_transforms.denormalize_lab(txt.cpu()),
                                        custom_transforms.denormalize_lab(skg.cpu()),
                                        texture_loc,
                                        color_space)
            tar_img = vis_image(custom_transforms.denormalize_lab(img.cpu()),
                                    color_space)
            #for ii in range(num_model):
            plt.subplot('1'+str(2*num_model)+str(2*i+1))
            plt.imshow(np.transpose(inp_img[0],(1, 2, 0)))
            plt.axis('off')
        #plt.figure()  
            plt.subplot('1'+str(2*num_model)+str(2*i+2))
            plt.imshow(np.transpose(out_img[0],(1, 2, 0)))
            plt.axis('off')
            temp_results.append([np.transpose(inp_img[0],(1, 2, 0)),np.transpose(out_img[0],(1, 2, 0))])
        results.append(temp_results)

        plt.show()
        datas.append(data_temps)